In [7]:
from ingestion import simulate_cdc_daily_parquets
import duckdb
import os
from dotenv import load_dotenv
load_dotenv()


ADLS_CONN_STRING = os.getenv("ADLS_CONN_STRING")


In [ ]:
def simulate_cdc_daily_parquets(
    container_name,
    tipo="yellow",
    ano=2022,
    mes=1,
    fonte_prefix="",
    cdc_prefix="cdc",
    data_col_map=None
):
    """
    Lê um arquivo Parquet do Azure Data Lake Storage (ADLS) no container especificado, filtra por dia e salva arquivos Parquet diários no caminho cdc/{tipo}/{ano}/{mes}/{date}.parquet.
    """
    from azure.storage.blob import BlobServiceClient
    import io

    blob_service_client = BlobServiceClient.from_connection_string(ADLS_CONN_STRING)
    container_client = blob_service_client.get_container_client(container_name)

    # Descobre o arquivo Parquet de origem no ADLS
    file_name = f"{tipo}_tripdata_{ano}-{mes:02d}.parquet"
    blob_path = f"source/{tipo}/{ano}/{file_name}"
    print(blob_path)
    # Baixa o arquivo Parquet diretamente na memória
    blob_data = container_client.download_blob(blob_path).readall()
    
    # Descobre a coluna de data/hora
    if data_col_map is None:
        data_col_map = {
            "yellow": "tpep_pickup_datetime",
            "green": "lpep_pickup_datetime"
        }
    datetime_col = data_col_map[tipo]

    # Descobre todos os dias presentes no arquivo
    import pandas as pd
    parquet_bytes = io.BytesIO(blob_data)
    df = pd.read_parquet(parquet_bytes)

    # Descobre todos os dias presentes no arquivo
    days = df[datetime_col].dt.date.unique()
    days = sorted(days)

    # Para cada dia, filtra e salva um parquet diário no caminho cdc/{tipo}/{ano}/{mes}/{date}.parquet
    for dia in days:
        dia_str = dia.strftime('%Y-%m-%d')
        out_blob_path = f"{cdc_prefix}/{tipo}/{ano}/{mes:02d}/{dia_str}.parquet"

        # Filtra os dados para o dia específico
        filtered_data = df[df[datetime_col].dt.date == dia]

        # Converte o DataFrame filtrado para Parquet em memória
        output_buffer = io.BytesIO()
        filtered_data.to_parquet(output_buffer, index=False)
        output_buffer.seek(0)

        # Envia para o blob no ADLS
        container_client.upload_blob(out_blob_path, output_buffer.getvalue(), overwrite=True)
        print(f"CDC diário gerado: {out_blob_path}")

simulate_cdc_daily_parquets(
    container_name="taxi",
    tipo="yellow",
    ano=2022,
    mes=1
)


source/yellow/2022/yellow_tripdata_2022-01.parquet
CDC diário gerado: cdc/yellow/2022/01/2008-12-31.parquet
CDC diário gerado: cdc/yellow/2022/01/2009-01-01.parquet
CDC diário gerado: cdc/yellow/2022/01/2021-12-31.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-01.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-02.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-03.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-04.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-05.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-06.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-07.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-08.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-09.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-10.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-11.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-12.parquet
CDC diário gerado: cdc/yellow/2022/01/2022-01-13.parquet
CDC diário gerado: cdc/yellow/2022/01